# Experiment 1

1. Create a random sat formula
* n = 10
* k = 3
* r = 4.0, 4.2, 4.4, 4.6


1. Check the number of sat clauses with the following solvers (mean over 10 times)
* Learning solver with greedy baseline trainable initial state
* Learning solver with greedy baseline and zeros initial state

In [1]:
import torch

from src.encoders import GCNEncoder
from src.decoders import RNNDecoder
from src.encoder_decoder import EncoderDecoder
from src.embeddings import BasicEmbedding, IdentityEmbedding
from src.baselines import BaselineRollout
from src.init_states import EncoderOutputState
from src.init_vars import IdentityEncoderOutputVar
from src.init_contexts import EncoderOutputContext

from src.generator import UniformGenerator
from src.run_solvers import random_solver, learning_solver

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import tensorboard as tb
from torch.utils.tensorboard import SummaryWriter

In [2]:
def run_model_zstate(formula, baseline):
    # Architecture
    cell = 'GRU'
    hidden_size = 128
    num_layers = 1
    dropout = 0
    clip_logits_c = 0 #(default: 0)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    ## Embeddings
    embedding_size = 32
    assignment_emb = BasicEmbedding(num_labels=3, embedding_size=embedding_size)
    variable_emb = IdentityEmbedding()
    context_emb = IdentityEmbedding()
    input_size = embedding_size * 3

    # Encoder
    encoder = GCNEncoder(
        embedding_size=embedding_size,
        device=device
    )

    decoder = RNNDecoder(input_size = input_size,
                        cell = cell,
                        assignment_emb = assignment_emb,
                        variable_emb = variable_emb,
                        context_emb = context_emb,
                        hidden_size = hidden_size,
                        num_layers = num_layers,
                        dropout = dropout,
                        clip_logits_c = clip_logits_c)
    ## Initializers
    init_dec_var = IdentityEncoderOutputVar()
    init_dec_context = EncoderOutputContext(aggregation="max")
    init_dec_state = EncoderOutputState(embedding_size, hidden_size, aggregation="mean")

    ## Network
    policy_network = EncoderDecoder(encoder=encoder,
                                    decoder=decoder,
                                    init_dec_var=init_dec_var,
                                    init_dec_context=init_dec_context,
                                    init_dec_state=init_dec_state)

    # Training hyperparameters
    num_episodes = 500
    accumulation_steps = 1
    lr = 1e-3
    #baseline = 
    entropy_weight = 0
    clip_val = 1
    verbose = 1

    variables = None

    # num_sat, history_loss, history_num_sat
    return learning_solver(formula=formula,
                        num_variables=num_variables,
                        variables=variables,
                        encoder=encoder,
                        decoder=decoder,
                        init_dec_var=init_dec_var,
                        init_dec_context=init_dec_context,
                        init_dec_state=init_dec_state, 
                        num_episodes=num_episodes,
                        accumulation_steps=accumulation_steps,
                        lr=lr,
                        device=device,
                        baseline=baseline,
                        entropy_weight=entropy_weight,
                        clip_val=clip_val,
                        verbose=verbose)


def run_model_tstate(formula, baseline):
    # Architecture
    cell = 'GRU'
    hidden_size = 128
    num_layers = 1
    dropout = 0
    clip_logits_c = 0 #(default: 0)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    ## Embeddings
    embedding_size = 32
    assignment_emb = BasicEmbedding(num_labels=3, embedding_size=embedding_size)
    variable_emb = IdentityEmbedding()
    context_emb = IdentityEmbedding()
    input_size = embedding_size * 3

    # Encoder
    encoder = GCNEncoder(
        embedding_size=embedding_size,
        device=device
    )

    decoder = RNNDecoder(input_size = input_size,
                        cell = cell,
                        assignment_emb = assignment_emb,
                        variable_emb = variable_emb,
                        context_emb = context_emb,
                        hidden_size = hidden_size,
                        num_layers = num_layers,
                        dropout = dropout,
                        clip_logits_c = clip_logits_c)
    ## Initializers
    init_dec_var = IdentityEncoderOutputVar()
    init_dec_context = EncoderOutputContext(aggregation="max")
    init_dec_state = EncoderOutputState(embedding_size, hidden_size, aggregation="mean")

    ## Network
    policy_network = EncoderDecoder(encoder=encoder,
                                    decoder=decoder,
                                    init_dec_var=init_dec_var,
                                    init_dec_context=init_dec_context,
                                    init_dec_state=init_dec_state)

    # Training hyperparameters
    num_episodes = 500
    accumulation_steps = 1
    lr = 1e-3
    #baseline = 
    entropy_weight = 0
    clip_val = 1
    verbose = 1

    variables = None

    # num_sat, history_loss, history_num_sat
    return learning_solver(formula=formula,
                        num_variables=num_variables,
                        variables=variables,
                        encoder=encoder,
                        decoder=decoder,
                        init_dec_var=init_dec_var,
                        init_dec_context=init_dec_context,
                        init_dec_state=init_dec_state, 
                        num_episodes=num_episodes,
                        accumulation_steps=accumulation_steps,
                        lr=lr,
                        device=device,
                        baseline=baseline,
                        entropy_weight=entropy_weight,
                        clip_val=clip_val,
                        verbose=verbose)

In [3]:
experiment_name = 'expgnn'
num_experiments = 10 #num of times each experiment is run

# CNF Formula
num_variables = 20  # Num variables
k = 3   # Clause size
r_list = [4.0, 4.2, 4.4] #Radious

global_step = 0
for _, r in enumerate(r_list):
    print(f'r = {r} ...')

    log_dir='outputs/' + experiment_name + '/runs/n' + str(num_variables) +'/'+str(r)
    writer = SummaryWriter(log_dir = log_dir)        

    # Create a sat generator
    sat_gen = UniformGenerator(min_n = num_variables,
                                max_n = num_variables,
                                min_k = k,
                                max_k = k,
                                min_r = r,
                                max_r = r)
    
    for _ in tqdm(range(num_experiments)):
            
        # Create a random sat formula
        n, r, m, formula = sat_gen.generate_formula()

        ##################################################################
        # Random solver                                                  #
        ##################################################################
        num_sat = random_solver(n, formula)
        writer.add_scalar('random', num_sat, global_step, new_style=True)


        ##################################################################
        # Learning model with 1-sampled basleine                         #
        ##################################################################
        baseline = BaselineRollout(1)
        num_sat, _,_ = run_model_zstate(formula, baseline)
        writer.add_scalar('learned_gb_zstate', num_sat, global_step, new_style=True)

        ##################################################################
        # Learning model with 2-sampled basleine                         #
        ##################################################################
        baseline = BaselineRollout(2)
        num_sat, _,_ = run_model_tstate(formula, baseline)
        writer.add_scalar('learning_gb_tstate', num_sat, global_step, new_style=True)

        global_step +=1
    
    writer.close()

r = 4.0 ...


  0%|          | 0/10 [00:00<?, ?it/s]

Episode [500/500], Mean Loss 0.0000,  Mean num sat 75.0000


 10%|█         | 1/10 [01:05<09:51, 65.70s/it]

Episode [500/500], Mean Loss 0.0000,  Mean num sat 75.0000
Episode [500/500], Mean Loss 3.5534,  Mean num sat 77.0000


...

Episode [500/500], Mean Loss 0.0000,  Mean num sat 78.0000
Episode [500/500], Mean Loss -20.4273,  Mean num sat 82.0000


 90%|█████████ | 9/10 [09:36<01:03, 63.90s/it]

Episode [500/500], Mean Loss 0.0000,  Mean num sat 81.0000
Episode [500/500], Mean Loss 0.0000,  Mean num sat 78.0000


100%|██████████| 10/10 [10:40<00:00, 64.03s/it]

Episode [500/500], Mean Loss 0.0000,  Mean num sat 78.0000


In [7]:
#Upload log_dir
#tensorboard dev upload --logdir 'outputs/exp0/runs/n20'

#from tensorboard experimental to dataframe
#https://tensorboard.dev/experiment/nGoxU6HWQMaduPCXvBpxgw/
experiment_id = '22ZkNiaJTKKaMhOztOWzoA/'
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Internal error. Please report this and specify reference code 4a5be3ee-cfcf-4233-83d7-e17718152738."
	debug_error_string = "{"created":"@1651464479.742563768","description":"Error received from peer ipv4:34.95.66.171:443","file":"src/core/lib/surface/call.cc","file_line":903,"grpc_message":"Internal error. Please report this and specify reference code 4a5be3ee-cfcf-4233-83d7-e17718152738.","grpc_status":13}"
>

In [ ]:
#Save dataframe as csv
csv_path = 'csv/sip_expgnn.csv'
df.to_csv(csv_path, index=False)

df = pd.read_csv(csv_path)
df

In [ ]:
df2 = df.rename(columns = {'run': 'radius'}, inplace = False)

df2['ratio'] = df2['value'] * 1/(df2['radius'].astype(float) * num_variables)

plt.figure(figsize=(16, 10))
sns.lineplot(x='radius', y='ratio', hue='tag', ci=20, data=df2, estimator='mean').set_title("Ratio of satisfied clauses")